In [9]:
# import import_ipynb
from dotenv import load_dotenv
import os
import googleapi as google
import folium as fo
from datetime import datetime
import requests
import json
from datetime import datetime
from IPython.display import display, Image, HTML
from ipywidgets import widgets, interact, interact_manual, Layout
import difflib
from folium.plugins import FloatImage


def config():
    load_dotenv()

#google api key
google_api_key = os.getenv("google_api_key")
#weather api key
weather_key = os.getenv("Weather_key")
#templete api key (Add text to icon of weather)
template_api_key = os.getenv("template_api_key")
template_id = os.getenv("template_id")

#Used to autocomplete the location a user is searching for
autocomplete_api_key = os.getenv("autocomplete_api_key")


load_dotenv()

True

In [12]:

#This function takes a weather api key and the city lat & long to find the weather data
def weather_data(weather_key, city_lat, city_lon):
    data = google.get_weather_data(weather_key, city_lat, city_lon)
    temp = round(data["current"]['temp'])
    icon = data["current"]['weather'][0]['icon']
    iconurl = f"http://openweathermap.org/img/wn/{icon}@2x.png"
    return iconurl, temp


# This fucntion takes the city name the wearher icon and current temp to create a picture compining them
def create_weather_img(city, iconurl, temp):
    url = f'https://api.apitemplate.io/v1/create?template_id={template_id}'
    data = {
        "overrides": [
                {
                    "name": "text_quote",
                    "text": f"{temp}° in {city}",
                    "textBackgroundColor": "#FFFFFF",
                    "color": "#40053D"
                },
                {
                    "name": "background-image",
                    "src": f'{iconurl}'

                },
                {
                    "name": "background-color",
                    "backgroundColor": "#460154"
                }

            ]
        }
    h = {
        "X-API-KEY": f"{template_api_key}"
    }

    response = requests.post(url, headers=h, json=data)
    response.raise_for_status()
    weather_img = response.json()
    img_url = weather_img['download_url_png']
    return img_url

# This function takes a text and geoapify to return locations that matche the text
def find_locations(apikey, text):
    url = 'https://api.geoapify.com/v1/geocode/autocomplete?'
    query = {
        "text": {text},
        "format": "json",
        "apiKey": apikey
    }

    response = requests.get(url, params=query)
    response.raise_for_status()
    response = response.json()
    return response



In [ ]:

# main fucntion
def main(place_type, city_lat, city_lon, city):
    #create the map zooming into the chosen city
    city_map = fo.Map(
    location=[city_lat, city_lon],
    zoom_start=12,
    )
    # get the temp and Icon-temp of the location
    iconurl, temp = weather_data(weather_key, city_lat, city_lon)
    #create a compined picture of the icon and temp to display in folium
    weather_img_url = create_weather_img(city, iconurl, temp)
    #create floating image og the current weather
    FloatImage(weather_img_url, bottom=68, left=82).add_to(city_map)
    # get the chosen places in the chosen city
    places = google.googlePlacesSearch(google_api_key, city_lat, city_lon, place_type, place_type)
    # if api function returns an error
    if places == 'error':
             display(HTML(f"<h3 Style='color: red'>! No {place_type} Found in {city}!<h3>"))

    else: # get details of each place[name, phone, address, hours and website]
        places[0]
        count = 0
        for place in places:
            count += 1
            place_id = place['place_id']
            name = place['name']
            place_lat = place['geometry']['location']['lat']
            place_lon = place['geometry']['location']['lng']
            coords = place_lat, place_lon
            # Get details of each place
            details = google.googlePlacesDetails(google_api_key, place_id)
            place_address = details["formatted_address"]
            open_or_closed = ''

            # if the places details contains opening hours the place has a weekly hours
            if "opening_hours" in details.keys():
                #current daytime
                now = datetime.now()
                #current day
                today = now.strftime("%A")
                #If the place is open or not [true or false]
                curr_status = details['opening_hours']['open_now']
                week_days = details['opening_hours']['weekday_text']
                #find the day that matches today in the weekly hours
                for day in week_days:
                    if day.startswith(today):
                        day_found = day
                        index_day = week_days.index(day_found)
                #Todays hours
                week_days = details['opening_hours']['weekday_text'][index_day]
                #if paces is open now
                if curr_status:
                    open_or_closed = f"<span style='color:#8AE608'>Open Now-</span></b> {week_days}</b>"

                else: #place is closed
                    open_or_closed = f"<span style='color:#E60C08'>Closed Now-</span></b> Opens on {today}"

            #no hours availible
            else:
                open_or_closed = f"<span style='color:#E60C08'>No hours Available</span></b>"
            # if the place has a website
            if 'website' in details.keys():
                website = f"<a href={details['website']} target='_blank'>{details['website']}</a>"

            else:
                website = f"<b>No Website Found</a></br>"

            if 'formatted_phone_number' in details.keys():
                 place_phone = details["formatted_phone_number"]
            else:
                 place_phone = "No Phone Number Found!"

            # create a pop containing place info
            info = fo.Popup( f"<p style='text-overflow: ellipsis;'><b>Name:</b> {name}</br>\
                <b>Phone #: </b>{place_phone}</br>\
                <b>Address: </b> <a href=https://www.google.com/maps/place/{place_address.replace(' ','')} target='_blank'>{place_address}</a></br>\
                <b>Hours: {open_or_closed}</br>\
                <b>Website: </b> {website}</br></p>", max_width=300,min_width=300)
            #find the matching icon in the icons files
            with open('icons.json', 'r') as ficons:
                icons = json.load(ficons)
            icon_url = icons[place_type]
            #create costume icon
            icon = fo.features.CustomIcon(icon_url,
                                              icon_size=(26))
            #add place marker on map
            marker = fo.Marker(location=coords, popup=info, tooltip='Click for more!', icon=icon )
            city_map.add_child(marker)
        display(HTML(f"<h4 style='color:#4793FA;'>Found {count} {place_type.title()} places in {city.title()}.......</h4>"))

        #display the map
        display(city_map)



In [ ]:
# All place types from google place types
place_types = "accounting airport amusement_park aquarium art_gallery atm bakery bank bar beauty_salon bicycle_store book_store bowling_alley bus_station cafe campground car_dealer car_rental car_repair car_wash casino cemetery church city_hall clothing_store convenience_store courthouse dentist department_store doctor drugstore electrician electronics_store embassy fire_station florist funeral_home furniture_store gas_station gym hair_care hardware_store hindu_temple home_goods_store hospital insurance_agency jewelry_store laundry lawyer library light_rail_station liquor_store local_government_office locksmith lodging meal_delivery meal_takeaway mosque movie_rental movie_theater moving_company museum night_club painter park parking pet_store pharmacy physiotherapist plumber police post_office primary_school real_estate_agency restaurant roofing_contractor rv_park school secondary_school shoe_store shopping_mall spa stadium storage store subway_station supermarket synagogue taxi_stand tourist_attraction train_station transit_station travel_agency university veterinary_care zoo"

# create a list of the places
all_places = []

for i in place_types.split():
    all_places.append(i)


interact_manual = interact_manual.options(manual_name='Select Location')
display(HTML("<div style='background-color:#8AE608; 'padding:10px'; 'width: 200px'><h3>Enter You Location</h3></div>"))

@interact(Search='')
def search_locations(Search):
    #found locations matching search
    found = []
    #if user starts to type
    if Search:
        if len(Search) < 4:
            display(HTML(f"<p style='color:#6A6B59;'>Type More Than 4 Chars For Suggestions!</p>"))
        else:
            #get autocomplete address from api
            found_locations = find_locations(autocomplete_api_key, Search)
            for i in found_locations['results']:
                found.append(i['formatted'])

    else:
        display(HTML(f"<p style='color:#98E691;'>Start Typing For Suggestions</p>"))

    @interact_manual(Place = widgets.Select(
            options=found,
            # rows=10,
            description='Places',
            layout=Layout(width='50%', height='100px'),
            disabled=False
        ))
    def search_places(Place):
        #if a place has been selected
        if Place:
            #get geo data of place
            geo_data = google.googleGeocode(google_api_key, Place)
            for i in found_locations['results']:
                if i['formatted'] == Place:
                    city = i['city']
            city_lat = geo_data['lat']
            city_lon = geo_data['lng']

            interact_manual1 = interact_manual.options(manual_name='Find Places')
            display(HTML(f"<div style='background-color:#02347D; padding:10px; color:white'><h3>Search For Places in {city.title()}-</h3><p> Sart Typing for Suggestions</p></div>"))

            @interact(Search='')

            # search to find matched api list
            def search_places(Search):
                found = ''
                #if place search has been set find the closest match
                if Search:
                    display(HTML(f"<p style='color:#7E8504;'>Found Places-</p>"))
                    found = difflib.get_close_matches(Search, all_places, cutoff=0.2)
                else:
                    display(HTML(f"<p style='color:#7E8504;'>Start Typing For Suggestions, EX- 'Restaurant'</p>"))

                @interact_manual1(Place = widgets.Select(
                    options=found,
                    # rows=10,
                    description='Place',
                    disabled=False,
                ))
                def place_details(Place):
                    #If a place has been chosen call main function to find details
                    if Place:
                        display(HTML(f"<h4 style='color:#BEAD06;'>Searching for {Place.title()}s in {city.title()}.....</h4>"))
                        main(Place, city_lat, city_lon, city)
                    else: #input is null
                        display(HTML(f"<h3 style='color:#E60C08'>!Enter a Proper Place To Search For !<h3>"))

        else: #input is null
            display(HTML(f"<h3 style='color:#E60C08'>! Invalid Input !<h3>"))
